# Importando libs

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, Dropdown
import plotly.express as px
import plotly.graph_objects as go
import os
from pathlib import Path
import re

# Lendo o dataset

In [2]:
try:   
   df_processed = pd.read_csv('../data/prouni_2005_2019_processed.csv')
except Exception as e:
    print(f"Error: {e}")

#df_processed.head()

In [3]:
lista_colunas = df_processed.columns
lista_colunas

Index(['ANO_CONCESSAO', 'INSTITUICAO', 'TIPO', 'MODALIDADE_ENSINO', 'CURSO',
       'TURNO', 'SEXO', 'RACA', 'DEFICIENTE_FISICO', 'REGIAO', 'UF',
       'MUNICIPIO', 'IDADE'],
      dtype='object')

# Gráficos

Funções para criar botão de salvar gráfico em png

In [4]:
def _limpar_nome_arquivo(name: str, max_len: int = 160) -> str:
    name = (name or "grafico").strip()
    name = re.sub(r"\s+", " ", name)
    name = re.sub(r'[\\/:*?"<>|]+', "_", name)  # proibidos no Windows
    name = name.strip(" ._")
    if not name:
        name = "grafico"
    return name[:max_len]

def salvar_plotly_png(fig, folder: str, default_title: str = "grafico"):
    folder_path = Path(folder)
    folder_path.mkdir(parents=True, exist_ok=True)

    title_text = getattr(getattr(getattr(fig, "layout", None), "title", None), "text", None) or default_title
    base = _limpar_nome_arquivo(str(title_text))

    outpath = folder_path / f"{base}.png"
    i = 2
    while outpath.exists():
        outpath = folder_path / f"{base.replace(' ', '_')}_{i}.png"
        i += 1  

    fig.write_image(str(outpath))
    return str(outpath)

def fazer_plotly_widget_com_salvar(
    plot_fn,
    controls: dict,
    folder: str,
    button_label: str = "Salvar PNG",
    default_title: str = "grafico",
    run_once: bool = True,
):
    state = {"fig": None}

    out_plot = widgets.Output()
    msg = widgets.HTML("")

    btn = widgets.Button(description=button_label, icon="save", tooltip="Salva o gráfico atual em PNG")

    def _renderizar(change=None):
        
        with out_plot:
            out_plot.clear_output()
            kwargs = {k: w.value for k, w in controls.items()}
            fig = plot_fn(**kwargs)
            state["fig"] = fig
            try:
                fig.show()
            except AttributeError:
                print("Nenhum gráfico para mostrar")
            except Exception as e:
                print(f"Erro ao mostrar gráfico: {e}")

    def _on_salvar(_):
        fig = state["fig"]
        if fig is None:
            msg.value = "<span style='color:#b00'>Ainda não existe gráfico para salvar.</span>"
            return
        try:
            path = salvar_plotly_png(fig, folder, default_title=str(default_title.replace(" ", "_")))
            msg.value = f"<span style='color:#060'>Salvo em: {path.replace(' ', '_')}</span>"
        except Exception as e:
            msg.value = f"<span style='color:#b00'>Erro ao salvar: {e}</span>"

    btn.on_click(_on_salvar)

    for w in controls.values():
        w.observe(_renderizar, names="value")

    ui = widgets.HBox([*controls.values(), btn, msg])

    if run_once:
        _renderizar()

    display(ui, out_plot)

    #return {"ui": ui, "out": out_plot, "button": btn, "message": msg, "state": state}

In [5]:
df_chars = df_processed.copy()

In [6]:

#lista_renomeada = [col.replace("_", " ").upper() for col in lista_colunas]
lista_renomeada = ['ANO DE CONCESSÃO', 'INSTITUIÇÃO', 'TIPO DE BOLSA','MODALIDADE DE ENSINO','CURSO','TURNO','SEXO','RAÇA','DEFICIENTE FÍSICO','REGIÃO','UF','MUNICÍPIO','IDADE']
df_chars.rename(columns=dict(zip(lista_colunas, lista_renomeada)), inplace=True)
df_chars.columns

Index(['ANO DE CONCESSÃO', 'INSTITUIÇÃO', 'TIPO DE BOLSA',
       'MODALIDADE DE ENSINO', 'CURSO', 'TURNO', 'SEXO', 'RAÇA',
       'DEFICIENTE FÍSICO', 'REGIÃO', 'UF', 'MUNICÍPIO', 'IDADE'],
      dtype='object')

## Número de bolsas por idade, tipo e gênero

In [7]:
anos = ['TODOS OS ANOS']
anos.extend(sorted(df_chars["ANO DE CONCESSÃO"].unique()))

dropdown_ano = widgets.Dropdown(
    options=anos,
    value=anos[0],
    description="ANO"
)

df_idade_genero = df_chars.loc[df_chars["IDADE"].between(10, 99)].copy()

bins = list(range(10, 99, 5))
labels = [f"{i}-{i+4}" for i in range(10, 94, 5)]

df_idade_genero["FAIXA DE IDADE"] = pd.cut(
    df_idade_genero["IDADE"],
    bins=bins,
    labels=labels,
    right=False
)

faixas = sorted(df_idade_genero["FAIXA DE IDADE"].dropna().unique().astype(str))

dropdown_faixa = widgets.Dropdown(
    options=["TODAS AS IDADES"] + faixas,
    value="TODAS AS IDADES",
    description="FAIXA DE IDADE"
)
dropdown_ano.style = {"description_width": "initial"}
dropdown_faixa.style = {"description_width": "initial"}

def total_idade_tipo_e_sexo(ano, faixa_idade):
    base = df_idade_genero[df_idade_genero["ANO DE CONCESSÃO"] == ano]

    if ano == 'TODOS OS ANOS':
        base = df_idade_genero

    if faixa_idade != "TODAS AS IDADES":
        base = base[base["FAIXA DE IDADE"].astype(str) == faixa_idade]

    agrupado = (
        base.groupby(["TIPO DE BOLSA", "SEXO"])
            .size()
            .reset_index(name="QUANTIDADE")
    )

    tabela = agrupado.pivot_table(
        index="TIPO DE BOLSA",
        columns="SEXO",
        values="QUANTIDADE",
        fill_value=0,
    ).reset_index()

    if "F" not in tabela.columns:
        tabela["F"] = 0
    if "M" not in tabela.columns:
        tabela["M"] = 0

    tabela["TOTAL"] = tabela["F"] + tabela["M"]

    tabela = tabela.rename(columns={
        "F": "FEMININO",
        "M": "MASCULINO"
    })
    
    if tabela.empty:
        print("Sem dados para esse filtro.")
        return

    titulo = f"Quantidade de bolsas por tipo e sexo em {ano.lower()}" if ano == 'TODOS OS ANOS' else f"Quantidade de bolsas por tipo e sexo em {ano}"
    
    fig = px.bar(
        tabela,
        x="TIPO DE BOLSA",
        y=["TOTAL", "FEMININO", "MASCULINO"],
        barmode="group",
        title=titulo,
        text_auto=True,
        color_discrete_map={
            "TOTAL": "darkgreen",
            "FEMININO": "#e83e8c",
            "MASCULINO": "#1f77b4"
        }
    )

    fig.update_traces(
    hovertemplate=
        "TIPO: %{x}<br>"
        + "QTDE: %{y:,.0f}".replace(",", ".")
        + "<extra></extra>",
        textposition="inside",
        texttemplate="%{y:,.0f}".replace(",", ".")
    )

    
    fig.update_layout(
        height=600,
        width=1000,
        xaxis_title="TIPO DE BOLSA",
        yaxis_title="QUANTIDADE DE BOLSAS",
        uniformtext_minsize=6,
        uniformtext_mode="hide",
        legend_title="LEGENDA",
        plot_bgcolor='darkgray',
        paper_bgcolor="white",
        font=dict(color="black")
    )

    return fig

dropdown_ano.layout = widgets.Layout(width="180px")
dropdown_faixa.layout = widgets.Layout(width="250px")

fazer_plotly_widget_com_salvar(
    plot_fn=total_idade_tipo_e_sexo,
    controls={"ano": dropdown_ano, "faixa_idade": dropdown_faixa},
    folder=f"../figures/{total_idade_tipo_e_sexo.__name__}",
    default_title="total_idade_tipo_e_sexo"
)

Output()

In [8]:
df_idade_sexo = df_chars.loc[df_chars["IDADE"].between(10, 99)].copy()

bins = list(range(10, 99, 5))
labels = [f"{i}-{i+4}" for i in range(10, 94, 5)]

df_idade_sexo["FAIXA DE IDADE"] = pd.cut(
    df_idade_sexo["IDADE"],
    bins=bins,
    labels=labels,
    right=False
)

faixas = sorted(df_idade_sexo["FAIXA DE IDADE"].dropna().unique().astype(str))

dropdown_faixa = widgets.Dropdown(
    options=["TODAS AS IDADES"] + faixas,
    value="TODAS AS IDADES",
    description="FAIXA DE IDADE"
)

tipos = sorted(df_chars["TIPO DE BOLSA"].unique())
dropdown_tipo = widgets.Dropdown(options=tipos, value=tipos[0], description="TIPO DE BOLSA")

dropdown_faixa.style = {"description_width": "initial"}
dropdown_tipo.style = {"description_width": "initial"}

def crescimento_idade_tipo_e_sexo(faixa_idade, tipo_bolsa):
    base = df_idade_sexo.copy()

    # Mapear gênero
    base["SEXO"] = base["SEXO"].map({
        "F": "FEMININO",
        "M": "MASCULINO"
    })

    if faixa_idade != "TODAS AS IDADES":
        base = base[base["FAIXA DE IDADE"].astype(str) == faixa_idade]

    if tipo_bolsa != "TODOS OS TIPOS":
        base = base[base["TIPO DE BOLSA"] == tipo_bolsa]
    
    agrupado = (
        base
        .groupby(["TIPO DE BOLSA", "SEXO","ANO DE CONCESSÃO"])
        .size()
        .reset_index(name="QUANTIDADE")
    )
    
    agrupado_total = (
        base
        .groupby(['ANO DE CONCESSÃO','TIPO DE BOLSA'])
        .size()
        .reset_index(name="QUANTIDADE")
    )

    agrupado_total['SEXO'] = 'TOTAL'
    
    agrupado = pd.concat([agrupado, agrupado_total])
    
    if agrupado.empty:
        print("Sem dados para esse filtro.")
        return

    titulo = f'{tipo_bolsa.capitalize().replace('Bolsa','Bolsas').replace('parcial','parciais de').replace('complementar','complementares de').replace('integral','integrais')} do Prouni : faixa de idade {faixa_idade.lower()} anos' if faixa_idade != "TODAS AS IDADES" else f'{tipo_bolsa.capitalize().replace('Bolsa','Bolsas').replace('parcial','parciais de').replace('complementar','complementares de').replace('integral','integrais')} do Prouni' 
    fig = px.line(
        agrupado,
        x="ANO DE CONCESSÃO",
        y="QUANTIDADE",
        color="SEXO",
        hover_data={
            "ANO DE CONCESSÃO": True,
            "TIPO DE BOLSA": True,
            "SEXO": True,
            "QUANTIDADE": ':.0f',
        },
        color_discrete_map={
            "FEMININO": "#e83e8c",
            "MASCULINO": "#1f77b4",
            'TOTAL': 'darkgreen'
        },
        title=titulo
    )
    
    
    fig.update_traces(mode="lines+markers")

    fig.update_layout(
        xaxis_tickangle=-45,
        xaxis_title="Ano de Concessão",
        yaxis_title="Quantidade",
        legend_title="Gênero",
        xaxis=dict(dtick=1),
        plot_bgcolor='darkgray',
        paper_bgcolor="white",
        font=dict(color="black")
    )
    return fig

fazer_plotly_widget_com_salvar(
    plot_fn=crescimento_idade_tipo_e_sexo,
    controls={"faixa_idade": dropdown_faixa,'tipo_bolsa': dropdown_tipo},
    folder=f"../figures/{crescimento_idade_tipo_e_sexo.__name__}",
    default_title="crescimento_idade_tipo_e_sexo"
)


Output()

In [9]:
anos = ['TODOS OS ANOS']
anos.extend(sorted(df_chars["ANO DE CONCESSÃO"].unique()))

dropdown_ano = widgets.Dropdown(
    options=anos,
    value=anos[0],
    description="ANO"
)

df_idade_sexo = df_chars.loc[df_chars["IDADE"].between(10, 70)].copy()

bins = list(range(10, 75, 5))
labels = [f"{i}-{i+4}" for i in range(10, 70, 5)]

df_idade_sexo["FAIXA DE IDADE"] = pd.cut(
    df_idade_sexo["IDADE"],
    bins=bins,
    labels=labels,
    right=False
)

# Agrupar por ano, faixa de idade e sexo
contagem = (
    df_idade_sexo
    .groupby(["ANO DE CONCESSÃO", "FAIXA DE IDADE", "SEXO"])
    .size()
    .reset_index(name="QTD_BOLSAS")
)

# Gráfico em Plotly com slider de ano e barras agrupadas por sexo
fig = px.bar(
    contagem,
    x="FAIXA DE IDADE",
    y="QTD_BOLSAS",
    color="SEXO",
    animation_frame="ANO DE CONCESSÃO",
    barmode="group",
    labels={
        "FAIXA_DE_IDADE": "Faixa de idade",
        "QTD_BOLSAS": "Quantidade de bolsas",
        "SEXO": "Sexo",
        "ANO DE CONCESSÃO": "Ano de concessão"
    },
    color_discrete_map={
        "F": "#e83e8c",
        "M": "#1f77b4"
    },
    title="Bolsas do Prouni por faixa de idade, ano de concessão e sexo"
)

fig.update_layout(
    xaxis_tickangle=-45,
    plot_bgcolor='darkgray',
    paper_bgcolor="white",
    font=dict(color="black")
)
fig.show()


C:\Users\felip\AppData\Local\Temp\ipykernel_14456\2671480094.py:25: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Número de bolsas por idade, tipo e raça

In [10]:

lista_racas = df_chars['RAÇA'].unique().tolist()
lista_racas

['Branca', 'Parda', 'Amarela', 'Nao Informada', 'Preta', 'Indigena']

In [11]:
pd.DataFrame(df_chars['RAÇA']).value_counts()

RAÇA         
Branca           1160045
Parda            1111553
Preta             340994
Amarela            47077
Nao Informada      23237
Indigena            2864
Name: count, dtype: int64

In [12]:
anos = ['TODOS OS ANOS']
anos.extend(sorted(df_chars["ANO DE CONCESSÃO"].unique()))

dropdown_ano = widgets.Dropdown(
    options=anos,
    value=anos[0],
    description="ANO"
)

df_idade_raca = df_chars.loc[df_chars["IDADE"].between(10, 99)].copy()

bins = list(range(10, 99, 5))
labels = [f"{i}-{i+4}" for i in range(10, 94, 5)]

df_idade_raca["FAIXA DE IDADE"] = pd.cut(
    df_idade_raca["IDADE"],
    bins=bins,
    labels=labels,
    right=False
)

faixas = sorted(df_idade_raca["FAIXA DE IDADE"].dropna().unique().astype(str))

dropdown_faixa = widgets.Dropdown(
    options=["TODAS AS IDADES"] + faixas,
    value="TODAS AS IDADES",
    description="FAIXA DE IDADE"
)
dropdown_ano.style = {"description_width": "initial"}
dropdown_faixa.style = {"description_width": "initial"}

def total_idade_tipo_e_raça(ano, faixa_idade):
    base = df_idade_raca[df_idade_raca["ANO DE CONCESSÃO"] == ano]

    if ano == 'TODOS OS ANOS':
        base = df_idade_raca

    if faixa_idade != "TODAS AS IDADES":
        base = base[base["FAIXA DE IDADE"].astype(str) == faixa_idade]

    agrupado = (
        base.groupby(["TIPO DE BOLSA", "RAÇA"])
            .size()
            .reset_index(name="QUANTIDADE")
    )

    tabela = agrupado.pivot_table(
        index="TIPO DE BOLSA",
        columns="RAÇA",
        values="QUANTIDADE",
        fill_value=0,
    ).reset_index()

    if "Branca" not in tabela.columns:
        tabela["Branca"] = 0
    if "Preta" not in tabela.columns:
        tabela["Preta"] = 0
    if "Parda" not in tabela.columns:
        tabela["Parda"] = 0
    if "Amarela" not in tabela.columns:
        tabela["Amarela"] = 0
    if "Indigena" not in tabela.columns:
        tabela["Indigena"] = 0
    if "Nao Informada" not in tabela.columns:
        tabela["Nao Informada"] = 0

    tabela["TOTAL"] = tabela['Branca'] + tabela['Preta'] + tabela['Parda'] + tabela['Amarela'] + tabela['Indigena'] + tabela['Nao Informada']

    tabela = tabela.rename(columns={
        'Branca' : 'BRANCA',
        'Preta' : 'PRETA',
        'Parda' : 'PARDA',
        'Amarela' : 'AMARELA',
        'Indigena' : 'INDÍGENA',
        'Nao Informada' : 'NÃO INFORMADA'
    })
    
    if tabela.empty:
        print("Sem dados para esse filtro.")
        return

    titulo = f"Quantidade de bolsas por tipo e raça em {ano.lower()}" if ano == 'TODOS OS ANOS' else f"Quantidade de bolsas por tipo e raça em {ano}"
    
    fig = px.bar(
        tabela,
        x="TIPO DE BOLSA",
        y=["TOTAL", "BRANCA", "PRETA", "PARDA", "AMARELA", "INDÍGENA", "NÃO INFORMADA"],
        barmode="group",
        title=titulo,
        text_auto=True,
        color_discrete_map={
            "TOTAL": "darkgreen",
            "BRANCA": "#EDE4E8",
            "PRETA": "#3F4243",
            "PARDA": "#A68467",
            "AMARELA": "#FFEA05",
            "INDÍGENA": "#F06D6D",
            "NÃO INFORMADA": "blue"
        }
    )

    fig.update_traces(
    hovertemplate=
        "TIPO: %{x}<br>"
        + "QTDE: %{y:,.0f}".replace(",", ".")
        + "<extra></extra>",
        textposition="outside",
        texttemplate="%{y:,.0f}".replace(",", ".")
    )

    
    fig.update_layout(
        height=600,
        width=1400,
        xaxis_title="TIPO DE BOLSA",
        yaxis_title="QUANTIDADE DE BOLSAS",
        uniformtext_minsize=6,
        uniformtext_mode="show",
        legend_title="RAÇA",
        plot_bgcolor='darkgray',
        paper_bgcolor="white",
        font=dict(color="black")
    )

    return fig

dropdown_ano.layout = widgets.Layout(width="180px")
dropdown_faixa.layout = widgets.Layout(width="250px")

fazer_plotly_widget_com_salvar(
    plot_fn=total_idade_tipo_e_raça,
    controls={"ano": dropdown_ano, "faixa_idade": dropdown_faixa},
    folder=f"../figures/{total_idade_tipo_e_raça.__name__}",
    default_title="total_idade_tipo_e_raça"
)

Output()

In [13]:
df_idade_raca = df_chars.loc[df_chars["IDADE"].between(10, 99)].copy()

bins = list(range(10, 99, 5))
labels = [f"{i}-{i+4}" for i in range(10, 94, 5)]

df_idade_raca["FAIXA DE IDADE"] = pd.cut(
    df_idade_raca["IDADE"],
    bins=bins,
    labels=labels,
    right=False
)

faixas = sorted(df_idade_raca["FAIXA DE IDADE"].dropna().unique().astype(str))

dropdown_faixa = widgets.Dropdown(
    options=["TODAS AS IDADES"] + faixas,
    value="TODAS AS IDADES",
    description="FAIXA DE IDADE"
)

tipos = sorted(df_chars["TIPO DE BOLSA"].unique())
dropdown_tipo = widgets.Dropdown(options=tipos, value=tipos[0], description="TIPO DE BOLSA")

dropdown_faixa.style = {"description_width": "initial"}
dropdown_tipo.style = {"description_width": "initial"}

def crescimento_idade_tipo_e_raca(faixa_idade, tipo_bolsa):
    base = df_idade_raca.copy()

    # Mapear gênero
    base["RAÇA"] = base["RAÇA"].map({
        'Branca' : 'BRANCA',
        'Preta' : 'PRETA',
        'Parda' : 'PARDA',
        'Amarela' : 'AMARELA',
        'Indigena' : 'INDÍGENA',
        'Nao Informada' : 'NÃO INFORMADA'
    })

    if faixa_idade != "TODAS AS IDADES":
        base = base[base["FAIXA DE IDADE"].astype(str) == faixa_idade]

    if tipo_bolsa != "TODOS OS TIPOS":
        base = base[base["TIPO DE BOLSA"] == tipo_bolsa]
    
    agrupado = (
        base
        .groupby(["TIPO DE BOLSA", "RAÇA","ANO DE CONCESSÃO"])
        .size()
        .reset_index(name="QUANTIDADE")
    )
    
    agrupado_total = (
        base
        .groupby(['ANO DE CONCESSÃO','TIPO DE BOLSA'])
        .size()
        .reset_index(name="QUANTIDADE")
    )

    agrupado_total['RAÇA'] = 'TOTAL'
    
    agrupado = pd.concat([agrupado, agrupado_total])
    
    if agrupado.empty:
        print("Sem dados para esse filtro.")
        return

    titulo = f'{tipo_bolsa.capitalize().replace('Bolsa','Bolsas').replace('parcial','parciais de').replace('complementar','complementares de').replace('integral','integrais')} do Prouni : faixa de idade {faixa_idade.lower()} anos' if faixa_idade != "TODAS AS IDADES" else f'{tipo_bolsa.capitalize().replace('Bolsa','Bolsas').replace('parcial','parciais de').replace('complementar','complementares de').replace('integral','integrais')} do Prouni' 
    fig = px.line(
        agrupado,
        x="ANO DE CONCESSÃO",
        y="QUANTIDADE",
        color="RAÇA",
        hover_data={
            "ANO DE CONCESSÃO": True,
            "TIPO DE BOLSA": True,
            "RAÇA": True,
            "QUANTIDADE": ':.0f',
        },
        color_discrete_map={
            "TOTAL": "darkgreen",
            "BRANCA": "#EDE4E8",
            "PRETA": "#3F4243",
            "PARDA": "#A68467",
            "AMARELA": "#FFEA05",
            "INDÍGENA": "#F06D6D",
            "NÃO INFORMADA": "blue"
        },
        title=titulo
    )
    
    
    fig.update_traces(mode="lines+markers")

    fig.update_layout(
        xaxis_tickangle=-45,
        xaxis_title="Ano de Concessão",
        yaxis_title="Quantidade",
        legend_title="Raça",
        xaxis=dict(dtick=1),
        plot_bgcolor='darkgray',
        paper_bgcolor="white",
        font=dict(color="black")
    )
    return fig

fazer_plotly_widget_com_salvar(
    plot_fn=crescimento_idade_tipo_e_raca,
    controls={"faixa_idade": dropdown_faixa,'tipo_bolsa': dropdown_tipo},
    folder=f"../figures/{crescimento_idade_tipo_e_raca.__name__}",
    default_title="crescimento_idade_tipo_e_raca"
)


Output()

## Crescimento de cada tipo de bolsa ao longo dos anos no país

In [14]:
# Paleta opcional por tipo
cores_por_tipo = {
    "BOLSA INTEGRAL": "#1f77b4",
    "BOLSA PARCIAL 50%": "#ff7f0e",
    "BOLSA COMPLEMENTAR 25%": "#2ca02c"
}

# Dropdowns
tipos = sorted(df_chars["TIPO DE BOLSA"].unique())
dropdown_tipo = widgets.Dropdown(options=tipos, value=tipos[0], description="TIPO DE BOLSA")

# PAÍS = agregado, Comparar regiões = várias linhas
regioes = ["PAÍS INTEIRO", "COMPARAR REGIÕES"] + sorted(df_chars["REGIÃO"].unique())

dropdown_regiao = widgets.Dropdown(options=regioes, value="PAÍS INTEIRO", description="REGIÃO")

modos = ["QUANTIDADE", "CRESCIMENTO PERCENTUAL"]
dropdown_modo = widgets.Dropdown(options=modos, value="QUANTIDADE", description="ESCALA")

dropdown_tipo.style = {"description_width": "initial"}
dropdown_regiao.style = {"description_width": "initial"}
dropdown_modo.style = {"description_width": "initial"}

def crescimento_bolsas_anos(tipo, regiao, modo):
    base = df_chars[df_chars["TIPO DE BOLSA"] == tipo].copy()
    if base.empty:
        print("Sem dados para esse tipo.")
        return
    # Cenário 1: PAÍS (agregado)
    if regiao == "PAÍS INTEIRO":
        dados = (
            base.groupby("ANO DE CONCESSÃO")
                .size()
                .reset_index(name="QUANTIDADE")
                .sort_values("ANO DE CONCESSÃO")
        )
        dados["TIPO DE BOLSA"] = tipo

        primeiro_valor = dados["QUANTIDADE"].iloc[0]
        if primeiro_valor == 0:
            dados["Crescimento_percentual"] = pd.NA
        else:
            dados["Crescimento_percentual"] = (dados["QUANTIDADE"] / primeiro_valor - 1) * 100

        if modo == "QUANTIDADE":
            metrica = "QUANTIDADE"
            y_label = "Quantidade de bolsas"
        else:
            metrica = "Crescimento_percentual"
            y_label = "Crescimento em relação ao primeiro ano (%)"

        cor_tipo = cores_por_tipo.get(tipo, "#1f77b4")

        fig = px.line(
            dados,
            x="ANO DE CONCESSÃO",
            y=metrica,
            markers=True,
            title=f"Evolução de {tipo} no país",
            color_discrete_sequence=[cor_tipo],
            hover_data={
                "ANO DE CONCESSÃO": True,
                "TIPO DE BOLSA": True,
                "QUANTIDADE": ':.0f',
                "Crescimento_percentual": ':.1f' if modo != "QUANTIDADE" else False,
            }
        )

    # Cenário 2: região específica
    elif regiao in base["REGIÃO"].unique():
        base = base[base["REGIÃO"] == regiao]
        if base.empty:
            print("Sem dados para essa região.")
            return

        dados = (
            base.groupby("ANO DE CONCESSÃO")
                .size()
                .reset_index(name="QUANTIDADE")
                .sort_values("ANO DE CONCESSÃO")
        )
        dados["TIPO DE BOLSA"] = tipo
        dados["REGIÃO"] = regiao

        primeiro_valor = dados["QUANTIDADE"].iloc[0]
        if primeiro_valor == 0:
            dados["Crescimento_percentual"] = pd.NA
        else:
            dados["Crescimento_percentual"] = (dados["QUANTIDADE"] / primeiro_valor - 1) * 100

        if modo == "QUANTIDADE":
            metrica = "QUANTIDADE"
            y_label = "Quantidade de bolsas"
        else:
            metrica = "Crescimento_percentual"
            y_label = "Crescimento em relação ao primeiro ano (%)"

        cor_tipo = cores_por_tipo.get(tipo, "#1f77b4")

        fig = px.line(
            dados,
            x="ANO DE CONCESSÃO",
            y=metrica,
            markers=True,
            title=f"Evolução de {tipo} na região {regiao}",
            color_discrete_sequence=[cor_tipo],
            hover_data={
                "ANO DE CONCESSÃO": True,
                "TIPO DE BOLSA": True,
                "REGIÃO": True,
                "QUANTIDADE": ':.0f',
                "Crescimento_percentual": ':.1f' if modo != "QUANTIDADE" else False,
            }
        )

    # Cenário 3: Comparar regiões
    elif regiao == "COMPARAR REGIÕES":
        dados = (
            base.groupby(["ANO DE CONCESSÃO", "REGIÃO"])
                .size()
                .reset_index(name="QUANTIDADE")
                .sort_values(["REGIÃO", "ANO DE CONCESSÃO"])
        )

        # Crescimento percentual por região
        def calc_crescimento(s):
            if s.iloc[0] == 0:
                return pd.Series([pd.NA] * len(s), index=s.index)
            return (s / s.iloc[0] - 1) * 100

        dados["Crescimento_percentual"] = (
            dados.groupby("REGIÃO")["QUANTIDADE"].transform(calc_crescimento)
        )

        if modo == "QUANTIDADE":
            metrica = "QUANTIDADE"
            y_label = "Quantidade de bolsas"
        else:
            metrica = "Crescimento_percentual"
            y_label = "Crescimento em relação ao primeiro ano de cada região (%)"

        fig = px.line(
            dados,
            x="ANO DE CONCESSÃO",
            y=metrica,
            color="REGIÃO",
            markers=True,
            title=f"Evolução de {tipo} comparando regiões",
            hover_data={
                "ANO DE CONCESSÃO": True,
                "REGIÃO": True,
                "QUANTIDADE": ':.0f',
                "Crescimento_percentual": ':.1f' if modo != "QUANTIDADE" else False,
            }
        )

    else:
        print("Opção de região inválida.")
        return
        
    titulo = f"{modo.lower().title()} de {tipo.lower().replace('bolsa', 'bolsas').replace('parcial', 'parciais').replace('integral', 'integrais de').replace('complementar', 'complementares de')} no {regiao.lower()}" if regiao != 'COMPARAR REGIÕES' else f"{modo.lower().title()} de {tipo.lower().replace('bolsa', 'bolsas').replace('parcial', 'parciais').replace('integral', 'integrais').replace('complementar', 'complementares')} comparando regiões"
    fig.update_layout(
        xaxis_title="ANO DE CONCESSÃO",
        yaxis_title=y_label,
        xaxis=dict(dtick=1),
        height=650,
        title = titulo
    )
    return fig

dropdown_tipo.layout = widgets.Layout(width="320px")
dropdown_regiao.layout = widgets.Layout(width="230px")
dropdown_modo.layout = widgets.Layout(width="280px")

fazer_plotly_widget_com_salvar(
    plot_fn=crescimento_bolsas_anos,
    controls={"tipo": dropdown_tipo, "regiao": dropdown_regiao, "modo": dropdown_modo},
    folder=f"../figures/{crescimento_bolsas_anos.__name__}",
    default_title="crescimento_bolsas_anos"
)

Output()

## 

## Mapa

In [15]:
import json
# Carrega o geojson local (sem internet)
with open("brazil-states.geojson", encoding="utf-8") as f:
   geojson = json.load(f)

# Dropdown de anos
anos = ['TODOS OS ANOS'] + sorted(df_chars["ANO DE CONCESSÃO"].unique())
dropdown_ano_mapa = widgets.Dropdown(
   options=anos,
   value=anos[0],
   description="ANO"
)

# Dropdown de tipos (com opção "Todos")
tipos = ["TODOS OS TIPOS"] + sorted(df_chars["TIPO DE BOLSA"].unique())
dropdown_tipo_mapa = widgets.Dropdown(
   options=tipos,
   value="TODOS OS TIPOS",
   description="TIPO"
)

dropdown_ano_mapa.style = {"description_width": "initial"}
dropdown_tipo_mapa.style = {"description_width": "initial"}

def mapa_bolsas_por_uf(ano, tipo):
   # Filtra por ano
   dados = df_chars[df_chars["ANO DE CONCESSÃO"] == ano].copy()
   
   if ano == "TODOS OS ANOS":
      dados = df_chars.copy()

   # Filtra por tipo, se não for "Todos"
   if tipo != "TODOS OS TIPOS":
      dados = dados[dados["TIPO DE BOLSA"] == tipo]
   
   if dados.empty:
      print("Sem dados para essa combinação de ano e tipo.")
      return
   
   # Agrupa por UF e REGIAO
   mapa_df = (
      dados.groupby(["UF", "REGIÃO"])
            .size()
            .reset_index(name="TOTAL")
            .sort_values("TOTAL", ascending=False)
   )

   # Título dinâmico
   if tipo == "TODOS OS TIPOS":
      if ano == "TODOS OS ANOS":
         titulo = f"Total de bolsas de todos os tipos concedidas por estado em todos os anos"
      else:
         titulo = f"Total de bolsas de todos os tipos concedidas por estado em {ano}"
   else:
      if ano == 'TODOS OS ANOS':
         titulo = f"Total de {tipo.lower().replace('bolsa', 'bolsas').replace('integral', 'integrais')} por estado em todos os anos"
      else:
         titulo = f"Total de {tipo.lower().replace('bolsa', 'bolsas').replace('integral', 'integrais')} por estado em {ano}"

   # Mapa coroplético por estado
   fig = px.choropleth(
      mapa_df,
      geojson=geojson,
      locations="UF",                  # sua coluna de UF, tipo "SC", "SP"
      featureidkey="properties.sigla", # campo de UF dentro do geojson
      color="TOTAL",
      color_continuous_scale="Turbo",
      hover_data={"UF": True, "REGIÃO": True, "TOTAL": True},
      title=titulo,
   )

   fig.update_geos(
      fitbounds="locations",
      visible=False
   )

   fig.update_layout(
      height=500,
      coloraxis_colorbar=dict(title="TOTAL DE BOLSAS"),
      coloraxis_showscale=True,
      margin={"r":0, "t":50, "l":0, "b":0},
      width=1000,
   ),

   return fig
   
dropdown_ano_mapa.layout = widgets.Layout(width="175px")
dropdown_tipo_mapa.layout = widgets.Layout(width="250px")

fazer_plotly_widget_com_salvar(
    plot_fn=mapa_bolsas_por_uf,
    controls={"ano": dropdown_ano_mapa, "tipo": dropdown_tipo_mapa},
    folder=f"../figures/{mapa_bolsas_por_uf.__name__}",
    default_title="mapa_bolsas_por_uf"
)

Output()